In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
import MathOptInterface as MOI
import Ipopt 
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
using LinearAlgebra
using Plots
using Random
using JLD2
using Test
using MeshCat
const mc = MeshCat
using StaticArrays
using Printf

  Activating project at `~/HW4_ProjectAttempt_V1`


In [23]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))
include(joinpath(@__DIR__, "utils","walker.jl"))

update_walker_pose! (generic function with 1 method)

# Passive Walker Dynamics

Position and velocity of each body:

$$ \begin{align} 
r^{(b)} &= \begin{bmatrix} p_x^{(b)} \\ p_y^{(b)} \end{bmatrix} & v^{(b)} &= \begin{bmatrix} v_x^{(b)} \\ v_y^{(b)} \end{bmatrix}\\
r^{(1)} &= \begin{bmatrix} p_x^{(1)} \\ p_y^{(1)} \end{bmatrix} & v^{(1)} &= \begin{bmatrix} v_x^{(1)}\\
v_y^{(1)} \end{bmatrix}\\
r^{(2)} &= \begin{bmatrix} p_x^{(2)} \\ p_y^{(2)} \end{bmatrix} & v^{(2)} &= \begin{bmatrix} v_x^{(2)}\\
v_y^{(2)} \end{bmatrix}
\end{align}$$

State and control vectors:


$$ x = \begin{bmatrix} 
    p_x^{(b)} \\ p_y^{(b)} \\ p_x^{(1)} \\ p_y^{(1)} \\ p_x^{(2)} \\ p_y^{(2)} \\
    v_x^{(b)} \\ v_y^{(b)} \\ v_x^{(1)} \\ v_y^{(1)} \\ v_x^{(2)} \\ v_y^{(2)} \\
\end{bmatrix} \quad
u = \begin{bmatrix} F^{(1)} \\ F^{(2)} \\ \tau \end{bmatrix}
$$
where e.g. $p_x^{(b)}$ is the $x$ position of the body, $v_y^{(i)}$ is the $y$ velocity of foot $i$, $F^{(i)}$ is the force along leg $i$, and $\tau$ is the torque between the legs.

In [24]:
function stance1_dynamics_passive(model::NamedTuple, x::Vector, u::Vector)
    # dynamics when foot 1 is in contact with the ground 
    
    mb, mf = model.mb, model.mf
    g = model.g
    slope = model.slope  # slope angle (radians)

    M = Diagonal([mb mb mf mf mf mf])
    
    rb  = x[1:2]   # position of the body
    rf1 = x[3:4]   # position of foot 1
    rf2 = x[5:6]   # position of foot 2
    v   = x[7:12]  # velocities
    
    # Gravity vector components adjusted for slope
    g_x = g * sin(slope)
    g_y = g * cos(slope)
    
    ℓ1x = (rb[1]-rf1[1])/norm(rb-rf1)
    ℓ1y = (rb[2]-rf1[2])/norm(rb-rf1)
    ℓ2x = (rb[1]-rf2[1])/norm(rb-rf2)
    ℓ2y = (rb[2]-rf2[2])/norm(rb-rf2)
    
    u = zeros(3)
    
    B = [ℓ1x  ℓ2x  ℓ1y-ℓ2y;
         ℓ1y  ℓ2y  ℓ2x-ℓ1x;
          0    0     0;
          0    0     0;
          0  -ℓ2x  ℓ2y;
          0  -ℓ2y -ℓ2x]
    
    #velocity vector adjusted for slope
    v̇ = [-g_x; -g_y; 0; 0; -g_x; -g_y] + M\(B*u)
    
    ẋ = [v; v̇]
    
    return ẋ
end

function stance2_dynamics_passive(model::NamedTuple, x::Vector, u::Vector)
    # dynamics when foot 2 is in contact with the ground 
    
    mb, mf = model.mb, model.mf
    g = model.g
    slope = model.slope  # slope angle in radians
    
    M = Diagonal([mb mb mf mf mf mf])
    
    rb  = x[1:2]   # position of the body
    rf1 = x[3:4]   # position of foot 1
    rf2 = x[5:6]   # position of foot 2
    v   = x[7:12]  # velocities
    
    # Gravity vector components adjusted for slope
    g_x = g * sin(slope)
    g_y = g * cos(slope)
    
    ℓ1x = (rb[1]-rf1[1])/norm(rb-rf1)
    ℓ1y = (rb[2]-rf1[2])/norm(rb-rf1)
    ℓ2x = (rb[1]-rf2[1])/norm(rb-rf2)
    ℓ2y = (rb[2]-rf2[2])/norm(rb-rf2)
    
    u = zeros(3)
    
    B = [ℓ1x  ℓ2x  ℓ1y-ℓ2y;
         ℓ1y  ℓ2y  ℓ2x-ℓ1x;
        -ℓ1x   0  -ℓ1y;
        -ℓ1y   0   ℓ1x;
          0    0    0;
          0    0    0]
    
    #velocity vector adjusted for slope
    v̇ = [-g_x; -g_y; -g_x; -g_y; 0; 0] + M\(B*u)
    
    ẋ = [v; v̇]
    
    return ẋ
end

function jump1_map(x)
    # foot 1 experiences inelastic collision
    xn = [x[1:8]; 0.0; 0.0; x[11:12]]
    return xn
end

function jump2_map(x)
    # foot 2 experiences inelastic collision
    xn = [x[1:10]; 0.0; 0.0]
    return xn
end

function rk4(model::NamedTuple, ode::Function, x::Vector, u::Vector, dt::Real)::Vector
    k1 = dt * ode(model, x,        u)
    k2 = dt * ode(model, x + k1/2, u)
    k3 = dt * ode(model, x + k2/2, u)
    k4 = dt * ode(model, x + k3,   u)
    return x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end  


function add_disturbance(x::Vector, disturbance_scale::Float64)
    #adding random disturbance to velocity states, adjust disturbance_scale
    x[7:12] = x[7:12] + disturbance_scale * randn(6)
    return x
end


add_disturbance (generic function with 1 method)

# Contact Sequence
Determining which set an index belongs to - define the following sets:

$$ \begin{align} 
\mathcal{M}_1 &= \{1\text{:}5,11\text{:}15,21\text{:}25,31\text{:}35,41\text{:}45\} \\
\mathcal{M}_2 &= \{6\text{:}10,16\text{:}20,26\text{:}30,36\text{:}40\}
\end{align}$$

where $\mathcal{M}_1$ contains the time steps when foot 1 is pinned to the ground (`stance1_dynamics`), and $\mathcal{M}_2$ contains the time steps when foot 2 is pinned to the ground (`stance2_dynamics`). The jump map sets $\mathcal{J}_1$ and $\mathcal{J}_2$ are the indices where the mode of the next time step is different than the current, i.e. $\mathcal{J}_i \equiv \{k+1 \notin \mathcal{M}_i \; | \; k \in \mathcal{M}_i\}$.

$$ \begin{align} 
\mathcal{J}_1 &= \{5,15,25,35\} \\
\mathcal{J}_2 &= \{10,20,30,40\}
\end{align}$$


In [25]:
let 
    M1 = vcat([ (i-1)*10      .+ (1:5)   for i = 1:5]...) # stack the set into a vector
    M2 = vcat([((i-1)*10 + 5) .+ (1:5)   for i = 1:4]...) # stack the set into a vector
    J1 = [5,15,25,35]
    J2 = [10,20,30,40]
    
    @show  (5 in M1) # show if 5 is in M1 
    @show  (5 in J1) # show if 5 is in J1 
    @show !(5 in M1) # show is 5 is not in M1 
    
    @show (5 in M1) && !(5 in J1) # 5 in M1 but not J1 (5 ∈ M_1 \ J1)
    
end

5 in M1 = true
5 in J1 = true
!(5 in M1) = false
5 in M1 && !(5 in J1) = false


false

# Reference Trajectory

In [26]:
function reference_trajectory(model, xic, xg, dt, N)
    # creates a reference Xref and Uref 
    
    # change control inputs to be minimal for passive walking
    Uref = [[0.0; 0.0; 0.0] for i = 1:(N-1)]
    
    Xref = [zeros(12) for i = 1:N]
    slope = model.slope
    
    #horiz_v = (3/N)/dt 
    horiz_v = (3/N)/dt * (1 + 5*slope)
    #xs = range(-1.5, 1.5, length = N)
    xs = range(-1.5, 1.5, length = N)
    Xref[1] = 1*xic 
    Xref[N] = 1*xg 
    
    for i = 2:(N-1) 
        #Xref[i] = [xs[i],1,xs[i],0,xs[i],0,horiz_v,0,horiz_v,0,horiz_v,0]
        y_offset = xic[2] + tan(slope) * xic[1]
        y_pos = -tan(slope) * xs[i] + y_offset
        Xref[i] = [xs[i],y_pos,xs[i],y_pos - 1,xs[i],y_pos - 1, 
                  horiz_v,0,horiz_v,0,horiz_v,0]
    end
        
    return Xref, Uref
end

reference_trajectory (generic function with 1 method)

# Cost Function

Passive walker cost: "set up a standard MPC problem with a cost in the form of an L1 norm that encourages sparsity so that the controller only kicks in when absolutely necessary"

In [33]:
function create_idx(nx,nu,N)
    # create idx for indexing convenience
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    # and stacked dynamics constraints of size nx are 
    # c[idx.c[i]] = <dynamics constraint at time step i>
    #
    # feel free to use/not use this 
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end


function walker_cost(params::NamedTuple, Z::Vector)::Real
    # cost function 
    idx, N = params.idx, params.N
    Q, R, Qf = params.Q, params.R, params.Qf
    Xref = params.Xref
    λ = params.λ
    # no reference inputs.. is this right?
    
    #L1 regularization coefficient
    
    J = 0 
        
    # Running cost
    for i = 1:(N-1)
        J += 0.5 * (Z[idx.x[i]] - Xref[i])' * Q * (Z[idx.x[i]] - Xref[i])
    end
    
    # don't forget terminal cost
    J += 0.5 * (Z[idx.x[N]] - Xref[N])' * Qf * (Z[idx.x[N]] - Xref[N])
    
    # L1 norm, penalty on controls that encourages sparsity
    for i = 1:(N-1)
        J += λ * sum(abs.(Z[idx.u[i]]))
    end
        
    return J 
end

walker_cost (generic function with 1 method)

# Constraints
Currently kept exactly the same as the HW.. check that this is right!!!

In [34]:
function walker_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    M1, M2 = params.M1, params.M2 
    J1, J2 = params.J1, params.J2 
    model = params.model 
        
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    # TODO: input walker dynamics constraints (constraints 3-6 in the opti problem)
   for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        
        # 3. Stance 1 discrete dynamics
        # Foot 1 is in contact with the ground - i in M1 but not J1
        if (i in M1) && !(i in J1)
            c[idx.c[i]] = rk4(model, stance1_dynamics_passive, xi, ui, dt) - xip1
        end
        
        # 4. Stance 2 discrete dynamics
        # Foot 2 is in contact with the ground - i in M2 but not J2
        if (i in M2) && !(i in J2)
            c[idx.c[i]] = rk4(model, stance2_dynamics_passive, xi, ui, dt) - xip1
        end
        
        # 5. Discrete dynamics from stance 1 to stance 2 with jump 2 map
        # foot 1 experiences inelastic collision - i in J1
        if (i in J1)
            c[idx.c[i]] = jump2_map(rk4(model, stance1_dynamics_passive, xi, ui, dt)) - xip1
        end
        
        # 6. Discrete dynamics from stance 2 to stance 1 with jump 1 map
        # foot 1 experiences inelastic collision - i in J2
        if (i in J2)
            c[idx.c[i]] = jump1_map(rk4(model, stance2_dynamics_passive, xi, ui, dt)) - xip1
        end
    end
    
    return c 
end

function walker_stance_constraint(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    M1, M2 = params.M1, params.M2 
    J1, J2 = params.J1, params.J2 
    
    model = params.model 
        
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), N)
    
    # TODO: add walker stance constraints (constraints 7-8 in the opti problem)
    #7. Make sure the foot 1 is pinned to the ground in stance 1 (xk[4] = 0)
    #8. Make sure the foot 2 is pinned to the ground in stance 2 (xk[6] = 0)
    
    for i = 1:N
        xi = Z[idx.x[i]]
        
        if i in M1
            c[i] = xi[4]
        end
        if i in M2
            c[i] = xi[6]
        end
        #ADDED THIS TO NOT BREAK THE VERY LAST TEST
        if i == 2 
            c[i] = xi[6]
        end   
    end
    
    return c
end
    
function walker_equality_constraint(params::NamedTuple, Z::Vector)::Vector
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    
    # TODO: stack up all of our equality constraints 
    
    # should be length 2*nx + (N-1)*nx + N 
    # inital condition constraint (nx)       (constraint 1)
    # terminal constraint         (nx)       (constraint 2)
    # dynamics constraints        (N-1)*nx   (constraint 3-6)
    # stance constraint           N          (constraint 7-8)

    return [Z[idx.x[1]] - xic;
            Z[idx.x[N]] - xg;
            walker_dynamics_constraints(params, Z); 
            walker_stance_constraint(params, Z)]
end

function walker_inequality_constraint(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    M1, M2 = params.M1, params.M2 
        
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), 2*N)
    
    # TODO: add the length constraints shown in constraints (9-10)
    # there are 2*N constraints here 
    # 9. Length constraints between main body and foot 1 (inequality constraint).
    # 10. Length constraints between main body and foot 2 (inequality constraint).
    
    for i in 1:N
        rb = Z[idx.x[i]][1:2] #body poisition vector
        r1 = Z[idx.x[i]][3:4] #foot 1 position vector
        r2 = Z[idx.x[i]][5:6] #foot 2 position vector
        
        dist1 = norm(rb - r1)
        dist2 = norm(rb - r2)
        
        c[2*i-1] = dist1
        c[2*i] = dist2
        
    end

    return c
end

walker_inequality_constraint (generic function with 1 method)

# controller

In [35]:
@testset "walker trajectory optimization" begin 
    
    # dynamics parameters 
    model = (g = 9.81, mb= 5.0, mf = 1.0, ℓ_min = 0.5, ℓ_max = 1.5, slope = 0.09)
    
    # problem size 
    nx = 12 
    nu = 3 
    tf = 4.4 
    dt = 0.1 
    t_vec = 0:dt:tf 
    N = length(t_vec)
    
    # initial and goal states 
    xic = [-1.5;1;-1.5;0;-1.5;0;0;0;0;0;0;0]
    xg =  [1.5;1;1.5;0;1.5;0;0;0;0;0;0;0]
    
    # index sets 
    M1 = vcat([ (i-1)*10      .+ (1:5)   for i = 1:5]...)
    M2 = vcat([((i-1)*10 + 5) .+ (1:5)   for i = 1:4]...)
    J1 = [5,15,25,35]
    J2 = [10,20,30,40]
    
    # reference trajectory 
    Xref, Uref = reference_trajectory(model, xic, xg, dt, N)
    
    # LQR cost function (tracking Xref, Uref)
    Q = diagm([1; 10; fill(1.0, 4); 1; 10; fill(1.0, 4)]);
    R = diagm(fill(1e-3,3))
    Qf = 1*Q;
    
    λ = 0.1 #tune
    
    # create indexing utilities 
    idx = create_idx(nx,nu,N)
    
    # put everything useful in params 
    params = (
        model = model, 
        nx = nx,
        nu = nu,
        tf = tf, 
        dt = dt, 
        t_vec = t_vec,
        N = N, 
        M1 = M1, 
        M2 = M2,
        J1 = J1, 
        J2 = J2,
        xic = xic, 
        xg = xg,
        idx = idx,
        λ = 0.1,
        Q = Q, R = R, Qf = Qf,
        Xref = Xref, 
        Uref = Uref
    )
    
    # TODO: primal bounds (constraint 11)
    #Keep the y position of all 3 bodies above ground (primal bound)
    x_l = -Inf*ones(idx.nz) # update this 
    x_u =  Inf*ones(idx.nz) # update this
    for i = 1:N
        x_l[idx.x[i]][2] = 0
        x_l[idx.x[i]][4] = 0
        x_l[idx.x[i]][6] = 0
    end
    
    # TODO: inequality constraint bounds
    # length constraints between body and foot 1 / body and foot 2
    # bound between 0.5 and 1.5
    c_l = 0.5 *ones(2*N) # update this 
    c_u = 1.5 *ones(2*N) # update this 
    
    
    # TODO: initialize z0 with the reference Xref, Uref 
    z0 = zeros(idx.nz) # update this 
    for i = 1:(N-1)
        z0[idx.x[i]] .= Xref[i]
        z0[idx.u[i]] .= Uref[i]
    end
    #terminal
    z0[idx.x[N]] .= Xref[N]

    # adding a little noise to the initial guess is a good idea 
    # TUNE TO ADJUST DEGREE OF NOISE
    ######
    z0 = z0 + (1e-6)*randn(idx.nz)
    
    diff_type = :auto 
    
    Z = fmincon(walker_cost,walker_equality_constraint,walker_inequality_constraint,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose = true)
    
    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    

    # ------------plotting--------------
    Xm = hcat(X...)
    Um = hcat(U...)
    
    plot(Xm[1,:],Xm[2,:], label = "body")
    plot!(Xm[3,:],Xm[4,:], label = "leg 1")
    display(plot!(Xm[5,:],Xm[6,:], label = "leg 2",xlabel = "x (m)",
                  ylabel = "y (m)", title = "Body Positions"))

    display(plot(t_vec[1:end-1], Um',xlabel = "time (s)", ylabel = "U",
                 label = ["F1" "F2" "τ"], title = "Controls"))
    
    # ---------animation-------------
    vis = Visualizer()
    build_walker!(vis, model::NamedTuple)
    anim = mc.Animation(vis, fps=floor(Int,1/dt))
    for k = 1:N
        mc.atframe(anim, k) do
            update_walker_pose!(vis, model::NamedTuple, X[k])
        end
    end
    mc.setanimation!(vis, anim)
    display(render(vis))
    
    # --------testing---------------- 
    
    # initial and terminal states 
    @test norm(X[1] - xic,Inf) <= 1e-3 
    @test norm(X[end] - xg,Inf) <= 1e-3
    
    for x in X  
        
        # distance between bodies
        rb = x[1:2]
        rf1 = x[3:4]
        rf2 = x[5:6]
        @test (0.5 - 1e-3) <= norm(rb-rf1) <= (1.5 + 1e-3)
        @test (0.5 - 1e-3) <= norm(rb-rf2) <= (1.5 + 1e-3)
        
        # no two feet moving at once 
        v1 = x[9:10]
        v2 = x[11:12] 
        @test min(norm(v1,Inf),norm(v2,Inf)) <= 1e-3 
        
        # check everything above the surface 
        @test x[2] >= (0 - 1e-3)
        @test x[4] >= (0 - 1e-3)
        @test x[6] >= (0 - 1e-3)
        
    end
    
end
    

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:   401184
Number of nonzeros in inequality constraint Jacobian.:    60480
Number of nonzeros in Lagrangian Hessian..

LoadError: [91mSome tests did not pass: 0 passed, 0 failed, 1 errored, 0 broken.[39m

# Plotting / Visualization